In [1]:
import pyvista
import lfricflux
import numpy
import mint

In [2]:
inputFile = '../data/gungho/original/lfric_diag.nc'
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [3]:
# create an empty vector field object
x, y = lf.getEdgeLonLat()
vf = lfricflux.VtkVectors(x, y, vector_field_name='velocity', cartesian=True)

In [4]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [5]:
# last dimension is number of edges
u.shape

(1, 30, 3072)

In [32]:
# set the vector field
vf.setField(u, v)
# set time = 0, elev = 0
vf.setSlice((0, 29))

In [33]:
xyz = vf.getPointArray()
npoints = xyz.shape[0]

In [34]:
# create a pyvista mesh object
pvmesh = pyvista.UnstructuredGrid(vf.getMesh())
pvmesh.point_data.set_vectors(vectors=vf.vxyz, name='vectors')
pvmesh.point_data.set_scalars(numpy.sqrt((vf.vxyz**2).sum(axis=1)), name='mag')
pvmesh.point_data.keys()

['vectors', 'mag']

In [53]:
print(pvmesh.point_data['vectors'].shape)

(3072, 3)


In [35]:
glyphs = pvmesh.glyph(orient="vectors", scale="mag", factor=1.0, geom=pyvista.Arrow())

In [36]:
p = pyvista.Plotter()
p.add_mesh(pyvista.Sphere(radius=0.95), color="grey", ambient=0.6, opacity=1.0, show_edges=False)
p.add_mesh(glyphs)
p.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:53622/index.html?ui=P_0x184acb970_5&reconnect=auto' style='width: …

In [52]:
vf.setSlice((0, 29))
vf.vxyz.shape
glyphs.point_data['GlyphVector'].shape
# glyphs.point_data['GlyphVector'][:] = vf.vxyz
# print(glyphs.point_data['GlyphVector'].sum())
# vf.setSlice((0, 0))
# glyphs.point_data['GlyphVector'][:] = vf.vxyz
# print(glyphs.point_data['GlyphVector'].sum())

(310272, 3)

In [46]:
p2 = pyvista.Plotter()
p2.add_mesh(pyvista.Sphere(radius=0.95), color="grey", ambient=0.6, opacity=1.0, show_edges=False)
p2.add_mesh(glyphs)
vf.setSlice((0, 0))
glyphs.point_data['GlyphVector'][:] = vf.vxyz
p2.show(jupyter_backend='trame')

ValueError: could not broadcast input array from shape (3072,3) into shape (310272,3)

In [39]:
vals = glyphs.GetPointData().GetAbstractArray('vectors')

In [40]:
print(vals)

None
